In [1]:
import os
import sys
from sys import path
sys.path.append(os.path.expanduser('~/albatros_analysis'))
import numpy as np 
import numba as nb
import time
from scipy import linalg
from scipy import stats
from matplotlib import pyplot as plt
from datetime import datetime as dt
from src.correlations import baseband_data_classes as bdc
from src.utils import baseband_utils as butils
from src.utils import orbcomm_utils as outils
from scipy.optimize import least_squares
import json
import coord_helper as ch
import random
import h5py
import scripts.xcorr.helper as h
import importlib

BDC is using numpy
BDC is using numpy


STANDARDIZED INITIAL VARBS

In [2]:
config_file_name1 = 'config_24_07.json'
config_file_name2 = 'config_25_07.json'
baseline_idx = 1 
working_directory = '/project/s/sievers/thomasb/mars_data_24/4bit'
day1 = '24_07'
day2 = '25_07'

In [6]:
with open(f"{working_directory}/{day2}/{config_file_name2}", "r") as f:
    config = json.load(f)
    dir_parents = []
    coords = []
    # unpack information from the json file
    # Call get_starting_index for all antennas except reference
    for i, (ant, details) in enumerate(config["antennas"].items()):
        if (i == 0) or (i ==baseline_idx):
            coords.append(details['coordinates'])
            dir_parents.append(details["path"])
    global_start_time = config["correlation"]["start_timestamp"]
    global_end_time = config["correlation"]["end_timestamp"]
    c_acclen = config["correlation"]["coarse_acclen"]
    v_acclen = config["correlation"]["vis_acclen"]
    visibility_window = config["correlation"]["visibility_window"]
    T_SPECTRA = config["correlation"]["point_PFB"] / config["correlation"]["sample_rate"]

print("Antenna Paths:", dir_parents)
print("Antenna Coordinates:", coords)
print("Visibility Accumulation Length", v_acclen)
print("Coarse Accumulation Length:", c_acclen)
print(global_start_time)

Antenna Paths: ['/scratch/s/sievers/mohanagr/mars1_2024/baseband/', '/scratch/s/sievers/mohanagr/mars2_2024/baseband/']
Antenna Coordinates: [[79.41718333333333, -90.76735, 189], [79.41721666666666, -90.75885, 176]]
Visibility Accumulation Length 30000
Coarse Accumulation Length: 1000000
1721883602


In [7]:
ref_coords = coords[0]
a2_coords = coords[1]

ref_path = dir_parents[0]
a2_path = dir_parents[1]

C_T_ACCLEN = c_acclen* T_SPECTRA
V_T_ACCLEN = v_acclen* T_SPECTRA

c_nchunks = int((visibility_window)/C_T_ACCLEN)
v_nchunks = int((visibility_window)/V_T_ACCLEN)

tle_path = outils.get_tle_file(global_start_time, "/project/s/sievers/mohanagr/OCOMM_TLES")

context = [visibility_window, T_SPECTRA, v_acclen, v_nchunks, ref_coords]
print(context)
print(global_start_time)

[1000, 1.6384e-05, 30000, 2034, [79.41718333333333, -90.76735, 189]]
1721883602


GET THE SAT PASSES LIST

In [9]:
with open(f"{working_directory}/{day2}/pulsedata_{global_start_time}_{global_end_time}.json", "r") as f:
    pulsedata = json.load(f)
    
    info = []
    reliable_offsets = []
    all_offsets = []

    for pulse_idx, details in enumerate(pulsedata[f"{global_start_time}"]["antenna 1"]):  #maybe generalize to all antenna at some point?
        
        #for now, ignore multi-sat pulses
        if len(details["sats_present"]) > 1: 
            continue

        pulse_info = []

        #times
        start_time = details["start"] * 5
        end_time = details["end"] * 5
        pulse_info.append([start_time, end_time, global_start_time])

        #chans and satIDs
        satIDs = list(details['sats_present'].keys())
        sats = {}
        rel = True
        for satID in satIDs:
            satinfo = details['sats_present'][satIDs[0]]
            channels = []
            for detection in satinfo:
                relative_offset = detection[1]  #just get any (in this case last) relative offset

                channels.append(detection[0])
                if detection[2][0] < 0.9:
                    rel = False
                
            sats[satID] = channels
        
        pulse_info.append(sats)

        #offsets
        total_offset = details["timestream_offset"]
        all_offsets.append(total_offset)
        if rel == True:
            reliable_offsets.append(total_offset)
        pulse_info.append([relative_offset, total_offset])
        pulse_info.append(tle_path)

        info.append(pulse_info)

print(info)
print(len(info))
print(all_offsets)
print(len(all_offsets))
print(reliable_offsets)


[[[2740, 3155, 1721883602], {'28654': [1836, 1837]}, [140766, -6224776], '/project/s/sievers/mohanagr/OCOMM_TLES/2024/202407/20240725.txt'], [[5360, 5415, 1721883602], {'33591': [1850]}, [79756, -6224751], '/project/s/sievers/mohanagr/OCOMM_TLES/2024/202407/20240725.txt'], [[6465, 6735, 1721883602], {'59051': [1836, 1837]}, [79731, -6224776], '/project/s/sievers/mohanagr/OCOMM_TLES/2024/202407/20240725.txt'], [[9005, 9200, 1721883602], {'28654': [1836, 1837]}, [79781, -6224726], '/project/s/sievers/mohanagr/OCOMM_TLES/2024/202407/20240725.txt'], [[12480, 12925, 1721883602], {'59051': [1836, 1837]}, [79731, -6224776], '/project/s/sievers/mohanagr/OCOMM_TLES/2024/202407/20240725.txt'], [[18510, 19030, 1721883602], {'59051': [1836, 1837]}, [103156, -6224776], '/project/s/sievers/mohanagr/OCOMM_TLES/2024/202407/20240725.txt'], [[23320, 23630, 1721883602], {'25338': [1841]}, [79705, -6224802], '/project/s/sievers/mohanagr/OCOMM_TLES/2024/202407/20240725.txt'], [[24535, 25080, 1721883602], {

RESOLVE OVERFLOW ISSUE

In [10]:
reloffs1, reloffs2 = ch.split_array(reliable_offsets, 100000)  #possibly need to check which value I need here though

if reloffs2 == []:
    specnumoffset = int(stats.mode(reloffs1)[0])
    for i in range(len(info)):
        info[i][2][1] = specnumoffset

else:
    specnumoffsets = [int(stats.mode(reloffs1)[0]), int(stats.mode(reloffs2)[0])]
    overflow_index = ch.get_overflow_index(all_offsets, 10000)
    corrected_offsets = []
    for i in range(len(all_offsets)):
        if i < overflow_index:
            corrected_offsets.append(specnumoffsets[0])
        if i >= overflow_index:
            corrected_offsets.append(specnumoffsets[1])
    for i in range(len(info)):
        info[i][2][1] = corrected_offsets[i]


In [11]:
print(info[15])

[[36555, 37095, 1721883602], {'59051': [1836, 1837, 1850]}, [81129, -6224776], '/project/s/sievers/mohanagr/OCOMM_TLES/2024/202407/20240725.txt']


CHECK PULSES ALL AT ONCE

In [12]:
observed_data = []
time_total = time.time()

for pulse_idx in range(len(info)):

    print(f"---------STARTING PULSE {pulse_idx}---------")

    #--------times-----
    relative_start_time = info[pulse_idx][0][0]
    relative_end_time = info[pulse_idx][0][1]
    global_start_time = info[pulse_idx][0][2]

    print("pulse start, relative:", relative_start_time)
    print("pulse end, relative  :", relative_end_time)
    t_start = global_start_time + relative_start_time
    print("pulse start, global:", t_start)

    pulse_duration_seconds = relative_end_time - relative_start_time
    print("pulse duration, seconds", pulse_duration_seconds)

    if pulse_duration_seconds > visibility_window:
        print("pulse longer than vis window. cutting shorter")
        t_end = t_start + visibility_window
    elif pulse_duration_seconds < visibility_window:
       print("pulse shorter than vis window, set vis length (secs) to:", pulse_duration_seconds + 20)
       t_end = global_start_time + relative_end_time + 20   #buffer for safety and pleasure

    pulse_duration_chunks = int((pulse_duration_seconds + 20) / (T_SPECTRA * v_acclen)  )
    print("pulse is this many chunks long", pulse_duration_chunks)


    #----get initialized information----
    files_a1, idx1 = butils.get_init_info(t_start, t_end, ref_path)
    files_a2, idx2 = butils.get_init_info(t_start, t_end, a2_path)

    print("Files A1:", files_a1)
    print("IDX1:", idx1)



    #-------set up channels-------
    channels = bdc.get_header(files_a1[0])["channels"].astype('int64')
    chanstart = np.where(channels == 1834)[0][0] 
    chanend = np.where(channels == 1852)[0][0]
    nchans=chanend-chanstart

    idx_correction = info[pulse_idx][2][0] - 100000
    if idx_correction>0:
        idx1 = idx1 + idx_correction
    else:
        idx2 = idx2 + np.abs(idx_correction)


    #need to use this later, but for each channel now
    #chan_bigidx = info[pulse_idx][2]
    #chanmap = channels[chanstart:chanend].astype(int)
    #chan_smallidx = np.where(chanmap == chan_bigidx)[0][0]


    #--------open object----------
    ant1 = bdc.BasebandFileIterator(files_a1, 0, idx1, v_acclen, nchunks= pulse_duration_chunks, chanstart = chanstart, chanend = chanend, type='float')
    ant2 = bdc.BasebandFileIterator(files_a2, 0, idx2, v_acclen, nchunks= pulse_duration_chunks, chanstart = chanstart, chanend = chanend, type='float')


    #--------get visibilities-----
    specnumoffset = info[pulse_idx][2][1]
    m1= ant1.spec_num_start
    m2= m1 - specnumoffset

    print(f"m1: {m1}, m2: {m2}, offset: {specnumoffset}")
    print("specnum range ant1:", ant1.spec_num_start, ant1.spec_num_start + pulse_duration_chunks * v_acclen)
    print("specnum range ant2:", ant2.spec_num_start, ant2.spec_num_start + pulse_duration_chunks * v_acclen)

    

    visibility_phased = np.zeros((pulse_duration_chunks,len(ant1.channel_idxs)), dtype='complex64')
    time_pulse=time.time()
    #print(f"--------- Processing Pulse Idx {pulse_idx} ---------")
    for i, (chunk1,chunk2) in enumerate(zip(ant1,ant2)):
            xcorr = ch.avg_xcorr_4bit_2ant_float(
                chunk1['pol0'], 
                chunk2['pol0'],
                chunk1['specnums'],
                chunk2['specnums'],
                m1+i*v_acclen,
                m2+i*v_acclen)
            visibility_phased[i,:] = np.sum(xcorr,axis=0)/v_acclen
            print("CHUNK", i, " has ", xcorr.shape[0], " rows")
            print(f"ON PULSE {pulse_idx}")
    print(f"DONE PULSE {pulse_idx}. TIME:", time.time()-time_pulse)
    visibility_phased = np.ma.masked_invalid(visibility_phased)

    observed_data.append(visibility_phased)

print("Done with everything. Time taken:", time.time() - time_total)

---------STARTING PULSE 0---------
pulse start, relative: 2740
pulse end, relative  : 3155
pulse start, global: 1721886342
pulse duration, seconds 415
pulse shorter than vis window, set vis length (secs) to: 435
pulse is this many chunks long 885
Files A1: ['/scratch/s/sievers/mohanagr/mars1_2024/baseband/17218/1721886318.raw', '/scratch/s/sievers/mohanagr/mars1_2024/baseband/17218/1721886362.raw', '/scratch/s/sievers/mohanagr/mars1_2024/baseband/17218/1721886406.raw', '/scratch/s/sievers/mohanagr/mars1_2024/baseband/17218/1721886451.raw', '/scratch/s/sievers/mohanagr/mars1_2024/baseband/17218/1721886495.raw', '/scratch/s/sievers/mohanagr/mars1_2024/baseband/17218/1721886540.raw', '/scratch/s/sievers/mohanagr/mars1_2024/baseband/17218/1721886584.raw', '/scratch/s/sievers/mohanagr/mars1_2024/baseband/17218/1721886628.raw', '/scratch/s/sievers/mohanagr/mars1_2024/baseband/17218/1721886673.raw', '/scratch/s/sievers/mohanagr/mars1_2024/baseband/17218/1721886717.raw', '/scratch/s/sievers/mo

SAVE EVERYTHING RIGHT AWAY

In [13]:
with h5py.File(f'{working_directory}/{day2}/visraw_bline{baseline_idx}_{global_start_time}.h5', 'w') as f:
    for pulse_idx, observed in enumerate(observed_data):
        pulse_array = f.create_dataset(f'{pulse_idx}_{info[pulse_idx][0][0]}', data=observed)

        pulse_array.attrs['start_time'] = info[pulse_idx][0][0]
        pulse_array.attrs['end_time'] = info[pulse_idx][0][1]
        pulse_array.attrs['global_start_time'] = info[pulse_idx][0][2]
        pulse_array.attrs['sats'] = json.dumps(info[pulse_idx][1])
        pulse_array.attrs['tle_path'] = info[pulse_idx][3]

    f.attrs["vis_window"] = visibility_window
    f.attrs["v_acclen"] = v_acclen
    f.attrs["T_SPECTRA"] = V_T_ACCLEN
    f.attrs["v_nchunks"] = v_nchunks
    f.attrs["ref_coords"] = ref_coords #necessary?
    f.attrs["baseline_idx"] = baseline_idx